# 1 Importing packages and the functions.py

In [ ]:
import os
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import getpass
import glob
import seaborn as sns
import functions
import lfp_pre_processing_functions
import power_functions
import coherence_functions
import spectrogram_plotting_functions
import plotting_styles
import scipy.stats
import mne_connectivity
importlib.reload(functions) #loads our custom made functions.py file
importlib.reload(spectrogram_plotting_functions)
importlib.reload(plotting_styles)

linestyle = plotting_styles.linestyles
colors = plotting_styles.colors

# 2 - Loading the data files

This code fetches the current 'user' by using getpass. Then it sets the basepath, loads the files and specifies the savepath. Note that the basepath, files and savepath need to be changed depending on where you have kept the files and where you want the results to be stored. In this case, I have set it up to be in a particular folder in my Dropbox account, which is stored locally.

In [ ]:
#Fetch the current user
user= (getpass.getuser())
print("Hello", user)


if user == 'CPLab':
    base='D:\\Dropbox\\CPLab'
else:
    base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
#Set the basepath, savepath and load the data files
files = glob.glob(base+'\\all_data_mat_unfiltered\\*.mat')
savepath = base+'\\results\\'
print("Base path:", base)
print("Save path:", savepath)
print(files)

all_bands_dict = {'total':[1,100], 'beta':[12,30], 'gamma':[30,80], 'theta':[4,12]}


In [ ]:
keyboard_dict={'98':'b','119':'w','120':'nc','49':'1','48':'0'} #specifying the map of keyboard annotations to their meanings.
all_bands={'total':[1,100],'beta':[12,30], 'gamma':[30,80], 'theta':[4,12]}
importlib.reload(lfp_pre_processing_functions) #Reloading the lfp_pre_processing_functions module to ensure we have the latest version
#files=[f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat_filtered\\20230615_dk6_BW_context_day1.mat', f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat\\20230626_dk6_BW_nocontext_day1.mat'] #This is just for testing purposes

#Initializing a few empty things to store data
events_codes_all = {}
compiled_data_all_epochs = []
compiled_data_list=[]
compiled_shuffled_data_list = []
baseline_lfp_all = []
normalization_comparison_all = []
baseline_dict = {}
for file in files: #Looping through data files
    
    ## Get the date, mouse_id and task from the file name
    base_name = os.path.basename(file)
    base_name, _ = os.path.splitext(base_name)
    date, mouse_id, task=lfp_pre_processing_functions.exp_params(base_name) #Using a custom made function [see functions.py]
    print(date, mouse_id, task)
    if task == 'nocontextday2' or task == 'nocontextos2':
        task = 'nocontext'
    if task =='nocontext':
        continue
    f=h5py.File(file, 'r')  ## Open the data file
    channels = list(f.keys()) ## Extract channels list from the data file
    print(base_name, channels)
    if not any("AON" in channel or "vHp" in channel for channel in channels):
        continue
    events,reference_electrode=lfp_pre_processing_functions.get_keyboard_and_ref_channels(f,channels)

    events_codes=np.array(events['codes'][0]) #saving the keyboard annotations of the events (door open, door close etc.)
    events_times=np.array(events['times'][0]) #saving when the events happened
    events_codes_all[base_name] = events_codes #saving the codes in a dictionary to be analyzed later for events other than the ones in our keyboard_dict map
    
    #Generating epochs from events (epochs are basically start of a trial and end of a trial)
    epochs=lfp_pre_processing_functions.generate_epochs_with_first_event(events_codes, events_times)

    # task Start time
    first_event=events_times[0]
    #finding global start and end time of all channels, since they start and end recordings at different times
    global_start_time, global_end_time=lfp_pre_processing_functions.find_global_start_end_times(f,channels)
    
    ## Reference electrode finding and padding
    reference_time = np.array(reference_electrode['times']).flatten()
    reference_value = np.array(reference_electrode['values']).flatten()
    padd_ref_data,padded_ref_time=lfp_pre_processing_functions.pad_raw_data_raw_time(reference_value,reference_time,global_start_time,global_end_time,sampling_rate=2000)

    for channeli in channels:
        if "AON" in channeli or  "vHp" in channeli :
            
            channel_id=channeli
            # Extracting raw data and time
            data_all=f[channeli]
            raw_data=np.array(data_all['values']).flatten()
            raw_time = np.array(data_all['times']).flatten()
            sampling_rate = 2000
            print(channel_id)
            print(raw_data.shape, raw_time.shape, sampling_rate)
            
            padded_data,padded_time=lfp_pre_processing_functions.pad_raw_data_raw_time(raw_data,raw_time,global_start_time,global_end_time,sampling_rate)
            ref_subtracted_data = padded_data - padd_ref_data # Subtracting the reference electrode data from the raw data
            
            #notch_filtered_data = lfp_pre_processing_functions.iir_notch(raw_data, sampling_rate, 60)
            
            def extract_baseline_data(data,time,first_event,sampling_rate):
                if first_event>2.0:
                    baseline_data=data[np.where(time>first_event)[0][0]-2*sampling_rate:np.where(time>first_event)[0][0]]
                else:
                    baseline_data=data[0:np.where(time>first_event)[0][0]]
                baseline_mean=np.mean(baseline_data)
                baseline_std=np.std(baseline_data)
                
                #baseline_data_norm=(baseline_data-baseline_mean)/baseline_std
                print('normalizing data')
                return baseline_data,time, baseline_mean, baseline_std
            
            # Extracting baseline data
            data_before, time, baseline_mean, baseline_std=extract_baseline_data(ref_subtracted_data, raw_time, first_event, sampling_rate)
            first_event_index=np.where(raw_time>first_event)[0][0]
            baseline_row=[base_name, mouse_id,task,channel_id,np.array(data_before)]
            baseline_lfp_all.append(baseline_row)
            baseline_dict[base_name] = baseline_row

In [ ]:
baseline_lfp_all_df=pd.DataFrame(baseline_lfp_all, columns=['base_name', 'mouse_id', 'task', 'channel_id', 'data_before'])
baseline_lfp_all_df['welch'] = baseline_lfp_all_df['data_before'].apply(lambda x: power_functions.apply_welch_transform(x))
baseline_lfp_all_df['line_power'] = baseline_lfp_all_df['welch'].apply(lambda x: power_functions.get_band_power(x, 58, 62))
baseline_lfp_all_df['total_power'] = baseline_lfp_all_df['welch'].apply(lambda x: power_functions.get_band_power(x, 1, 100))
baseline_lfp_all_df['line_power_ratio'] = baseline_lfp_all_df['line_power'] / baseline_lfp_all_df['total_power']
baseline_lfp_all_df = baseline_lfp_all_df.sort_values(by=['mouse_id','task'])
baseline_lfp_final = baseline_lfp_all_df[['base_name', 'mouse_id', 'task', 'channel_id', 'line_power_ratio']]
#baseline_lfp_final=baseline_lfp_final.sort_values(by=['line_power_ratio'], ascending=False)
baseline_lfp_final = baseline_lfp_final.reset_index(drop=True)
baseline_lfp_final.to_csv(os.path.join(savepath, 'baseline_channel_selection_melted.csv'))


In [ ]:
channel_list= ['LFP1_AON', 'LFP2_AON', 'LFP3_AON', 'LFP4_AON', 'LFP1_vHp', 'LFP2_vHp']
base_names = baseline_lfp_all_df['base_name'].unique()
print(base_names)

In [ ]:

for base_namei in base_names[0]:
    fig, axs = plt.subplots(6, 1, figsize=(10, 6), sharex=True, sharey=True)
    fig.suptitle(base_namei)
    baseline_lfp_all_df_base = baseline_lfp_all_df[baseline_lfp_all_df['base_name'] == base_namei]
    for axi,channel_id in enumerate(channel_list):
        channel_data = baseline_lfp_all_df_base[baseline_lfp_all_df_base['channel_id'] == channel_id]
        if channel_data.empty:
            continue
        welch_data = channel_data['welch'].values[0]
        frequency = np.linspace(0, 1000, len(welch_data))
        ax= axs[axi]
        ax.plot(frequency, welch_data, label=channel_id)
        ax.set_title(channel_id)

In [ ]:
baseline_lfp_all_df = baseline_lfp_all_df.sort_values(by=['mouse_id', 'task', 'base_name'])
final_table = baseline_lfp_all_df.pivot(index=['base_name'], columns='channel_id', values='line_power_ratio')
final_table = final_table.reindex(
    baseline_lfp_all_df.drop_duplicates('base_name').sort_values(['mouse_id', 'task', 'base_name'])['base_name']
)
final_table = final_table[['LFP1_AON', 'LFP2_AON', 'LFP3_AON', 'LFP4_AON', 'LFP1_vHp', 'LFP2_vHp']]
final_table_melted = final_table.reset_index().melt(id_vars='base_name', var_name='channel_id', value_name='line_power_ratio')
final_table.to_csv(os.path.join(savepath, 'baseline_channel_selection.csv'))

In [ ]:
manual_check=pd.read_excel(os.path.join(savepath, 'filecheck_8325.xlsx'))
manual_check=manual_check[['base_name', 'LFP1AON', 'LFP2AON', 'LFP3AON', 'LFP4AON', 'LFP1vHC', 'LFP2vHC']]
manual_check = manual_check.rename(columns={
    'LFP1AON': 'LFP1_AON', 
    'LFP2AON': 'LFP2_AON', 
    'LFP3AON': 'LFP3_AON', 
    'LFP4AON': 'LFP4_AON', 
    'LFP1vHC': 'LFP1_vHp', 
    'LFP2vHC': 'LFP2_vHp'
})
manual_check = manual_check.set_index('base_name')
manual_check.to_csv(os.path.join(savepath, 'manual_check_channel_selection.csv'))
